In [21]:
from datasets import load_dataset

# loading CIFAR10 image dataset from HuggingFace
train_dataset = load_dataset('cifar10', split='train')
test_dataset = load_dataset('cifar10', split='test')

num_classes = len(set(train_dataset['label'])) # get number of labels in dataset

In [22]:
import torchvision.transforms as transforms

# resize all images to a fixed size 32 and normalize using suitable mean and standard deviation for the dataset
img_size = 32
mean = [0.4670, 0.4735, 0.4662]
std = [0.2496, 0.2489, 0.2521]
preprocess = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

from tqdm.auto import tqdm

train_inputs = []

def preprocess_dataset(dataset):
    inputs = []
    for record in tqdm(dataset):
        image, label = record['img'], record['label']
        if image.mode == 'L':
            image = image.convert('RGB')
        input_tensor = preprocess(image)
        inputs.append([input_tensor, label])
    return inputs

# preprocess training and test dataset
train_inputs = preprocess_dataset(train_dataset)
test_inputs = preprocess_dataset(test_dataset)

  0%|          | 0/50000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [26]:
from torch.utils.data import DataLoader

batch_size = 128

dataloader_train = DataLoader(train_inputs, batch_size=batch_size, shuffle=True)
dataloader_test = DataLoader(test_inputs, batch_size=batch_size, shuffle=False)

In [32]:
import torch.nn as nn

# constructing ConvNet
class ConvNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()

        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2)

        self.conv_layer2 = nn.Conv2d(in_channels=64, out_channels=192, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2)

        self.conv_layer3 = nn.Conv2d(in_channels=192, out_channels=384, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()

        self.conv_layer4 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, padding=1)
        self.relu4 = nn.ReLU()

        self.conv_layer5 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.relu5 = nn.ReLU()
        self.pool5 = nn.MaxPool2d(kernel_size=3, stride=2)

        self.dropout6 = nn.Dropout(p=0.5)
        self.fc6 = nn.Linear(in_features=2304, out_features=512)
        self.relu6 = nn.ReLU()

        self.dropout7 = nn.Dropout(p=0.5)
        self.fc7 = nn.Linear(in_features=512, out_features=256)
        self.relu7 = nn.ReLU()

        self.fc8 = nn.Linear(in_features=256, out_features=num_classes)

    def forward(self, x):
        x = self.conv_layer1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv_layer2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = self.conv_layer3(x)
        x = self.relu3(x)

        x = self.conv_layer4(x)
        x = self.relu4(x)

        x = self.conv_layer5(x)
        x = self.relu5(x)
        x = self.pool5(x)

        x = x.reshape(x.size(0), -1)

        x = self.dropout6(x)
        x = self.fc6(x)
        x = self.relu6(x)

        x = self.dropout7(x)
        x = self.fc7(x)
        x = self.relu7(x)

        x = self.fc8(x)

        return x

In [34]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = ConvNet(num_classes=num_classes).to(device)

loss_func = nn.CrossEntropyLoss()
learning_rate = 0.008
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(dataloader_train):
        # move tensors to configured device
        images, labels = images.to(device), labels.to(device)

        # forward propagation
        outputs = model(images)
        loss = loss_func(outputs, labels)

        # backward propagation and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # check validation loss and acc of each epoch
    with torch.no_grad():
        # switch model to eval (not train) model
        model.eval()
        total, correct = 0, 0
        all_test_loss = []
        for images, labels in dataloader_test:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            total += labels.size(0)
            # calc predictions
            predicted = torch.argmax(outputs, dim=1)
            # calc actual values
            correct += (predicted == labels).sum().item()
            # calc the loss
            all_test_loss.append(loss_func(outputs, labels).item())

        # calc test-loss
        mean_test_loss = sum(all_test_loss) / len(all_test_loss)
        # calc test-accuracy
        mean_test_acc = 100 * (correct / total)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Test-Loss: {mean_test_loss:.4f}, Test-Accuracy: {mean_test_acc:.1f}%')

# save to file
torch.save(model, 'cnn.pt')

Epoch [1/100], Loss: 2.3052, Test-Loss: 2.3024, Test-Accuracy: 11.5%
Epoch [2/100], Loss: 2.3021, Test-Loss: 2.3014, Test-Accuracy: 15.0%
Epoch [3/100], Loss: 2.3021, Test-Loss: 2.3000, Test-Accuracy: 12.7%
Epoch [4/100], Loss: 2.2980, Test-Loss: 2.2966, Test-Accuracy: 15.9%
Epoch [5/100], Loss: 2.2784, Test-Loss: 2.2809, Test-Accuracy: 19.7%
Epoch [6/100], Loss: 2.1417, Test-Loss: 2.0701, Test-Accuracy: 21.1%
Epoch [7/100], Loss: 1.9233, Test-Loss: 1.9582, Test-Accuracy: 24.4%
Epoch [8/100], Loss: 1.9183, Test-Loss: 1.9837, Test-Accuracy: 24.9%
Epoch [9/100], Loss: 1.8472, Test-Loss: 1.9937, Test-Accuracy: 24.9%
Epoch [10/100], Loss: 1.8956, Test-Loss: 1.7262, Test-Accuracy: 34.0%
Epoch [11/100], Loss: 1.7636, Test-Loss: 1.6831, Test-Accuracy: 35.9%
Epoch [12/100], Loss: 1.6355, Test-Loss: 1.6761, Test-Accuracy: 36.1%
Epoch [13/100], Loss: 1.6145, Test-Loss: 1.5939, Test-Accuracy: 38.6%
Epoch [14/100], Loss: 1.6925, Test-Loss: 1.5753, Test-Accuracy: 40.8%
Epoch [15/100], Loss: 1.4327,